In [2]:
import pandas as pd
import math
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt

In [3]:
"""假日預測
找出此生活圈（車站為主）的熱門地點
與此uid的家距離n個網格內的地點算一個生活圈
找出家在這個生活圈的所有uid
找出這些uid在假日去的地點按照比率排序

規律人與不規律人的分析
若一個人同一時段的地點重複次數超過一定比率，則為規律人，否則為不規律人
規律人就填他常去的地點
不規律人就填這個生活圈常去的地點

通勤時線性插值"""

'假日預測\n找出此生活圈（車站為主）的熱門地點\n與此uid的家距離n個網格內的地點算一個生活圈\n找出家在這個生活圈的所有uid\n找出這些uid在假日去的地點按照比率排序\n\n規律人與不規律人的分析\n若一個人同一時段的地點重複次數超過一定比率，則為規律人，否則為不規律人\n規律人就填他常去的地點\n不規律人就填這個生活圈常去的地點\n\n通勤時線性插值'

In [4]:
# 計算歐式距離
def euclidean_distance(loc1, loc2):
    """計算兩個地點之間的歐式距離"""
    dist=math.sqrt((loc1[0] - loc2[0])**2 + (loc1[1] - loc2[1])**2)
    #print("dist=",dist)
    return dist
# 判斷是否為假日
def is_weekend(day):
    # 星期日：day % 7 == 0，星期六：day % 7 == 6
    return (day % 7 == 0) or (day % 7 == 6)
# 找常去地點 p1, p2, p3
def find_places(group, morning_earliest, morning_latest, afternoon_earliest, afternoon_latest, evening_earliest, evening_latest):
    # 找出第一個常去地點 (上午)
    # 篩選上午時間段
    if morning_earliest is not None and morning_latest is not None:
        p1_time_group = group[(group['t'] >= morning_earliest) & (group['t'] <= morning_latest)]
    else:
        p1_time_group = group  # 如果條件為 None，則不篩選
    p1_location_counter = Counter(zip(p1_time_group['x'], p1_time_group['y']))
    p1_most_common = p1_location_counter.most_common(1)
    # 篩選下午時間段
    if afternoon_earliest is not None and afternoon_latest is not None:
        p2_time_group = group[(group['t'] >= afternoon_earliest) & (group['t'] <= afternoon_latest)]
    else:
        p2_time_group = group  # 如果條件為 None，則不篩選

    # 找出出現最多次的地點 (下午)
    p2_location_counter = Counter(zip(p2_time_group['x'], p2_time_group['y']))
    p2_most_common = p2_location_counter.most_common(1)

    # 篩選晚上/凌晨時間段
    if evening_earliest is not None and morning_earliest is not None:
        p3_time_group = group[(group['t'] >= evening_earliest) | (group['t'] <= morning_earliest)]
    else:
        p3_time_group = group  # 如果條件為 None，則不篩選

    # 找出出現最多次的地點 (晚上/凌晨)
    p3_location_counter = Counter(zip(p3_time_group['x'], p3_time_group['y']))
    p3_most_common = p3_location_counter.most_common(1)

    # 檢查是否找到家與工作地，避免出現空列表的情況
    p1 = p1_most_common[0][0] if p1_most_common else None
    p2 = p2_most_common[0][0] if p2_most_common else None
    p3 = p3_most_common[0][0] if p3_most_common else None
    
    return p1, p2, p3

# 判斷是否為通勤
def is_commuting(row1, row2):
    dist = euclidean_distance((row1['x'], row1['y']), (row2['x'], row2['y']))
    if dist >= 6:
        return True
    else:
        return False
# 分析通勤模式
def analyze_commuting_pattern(group):
    commuting_periods = []
    for i in range(len(group) - 1):
        if is_commuting(group.iloc[i], group.iloc[i+1]):
            commuting_periods.append(group.iloc[i]['t'])  # 記錄通勤時間段
    return commuting_periods
def filter_non_continuous_periods(commuting_periods, threshold=2):
    """
    過濾掉不連續的時間段，相鄰時間段之間的差距必須小於等於 threshold，否則被踢除。
    """
    if(commuting_periods!=[]):
        filtered_periods = [commuting_periods[0]]  # 保留第一個時間段
        for i in range(1, len(commuting_periods)):
            if abs(commuting_periods[i] - commuting_periods[i - 1]) <= threshold:
                filtered_periods.append(commuting_periods[i])
        return filtered_periods
    else:
        return []
#分成早上、下午、晚上通勤
def split_time_slot(commuting_periods):
    morning_commuting = []
    afternoon_commuting=[]
    evening_commuting = []
    
    # 將通勤時段分類為早上下午和晚上
    for i in commuting_periods:
        if 12 <= i < 24:
            morning_commuting.append(i)
        elif 24 <= i < 34:
            afternoon_commuting.append(i)
        elif 34 <= i < 44:
            evening_commuting.append(i)
    
    # 計算早上和晚上的時間段頻率
    morning_counter = Counter(morning_commuting)
    afternoon_counter = Counter(afternoon_commuting)
    evening_counter = Counter(evening_commuting)
    print("morning_counter=",morning_counter)
    print("afternoon_counter=",afternoon_counter)
    print("evening_counter=",evening_counter)
    # 找出最常見的前四個時間段
    morning_top4 = [time for time, count in morning_counter.most_common(4)]
    afternoon_top4 = [time for time, count in afternoon_counter.most_common(4)]
    evening_top4 = [time for time, count in evening_counter.most_common(4)]
    # 過濾掉不連續的時間段
    morning_top4_filtered = filter_non_continuous_periods(sorted(morning_top4))
    afternoon_top4_filtered = filter_non_continuous_periods(sorted(afternoon_top4))
    evening_top4_filtered = filter_non_continuous_periods(sorted(evening_top4))

    return morning_top4_filtered,afternoon_top4_filtered, evening_top4_filtered
def linear_interpolate(location1, location2, t, t_start, t_end):
    # 計算比例
    if t_end == t_start:
        return location1  # 避免除以0的情況

    ratio = (t - t_start) / (t_end - t_start)
    x = location1[0] + ratio * (location2[0] - location1[0])
    y = location1[1] + ratio * (location2[1] - location1[1])
    #print("ratio=",ratio,"x=",x,"y=",y)
    return round(x), round(y)

In [12]:
# 計算兩個點 (x1, y1) 和 (x2, y2) 之間的距離
def calculate_distance(x1, y1, x2, y2):
    return sqrt((x2 - x1)**2 + (y2 - y1)**2)

def classify_person(df_person, threshold=0.5):
    """
    1. 統計過去60天中的每個週末（禮拜六和禮拜日）的所有時間段 t，
       舉例：假設 8 個週末中的 t=10 時段，有 4 個週末的 t=10 都是在同一地點，
       那麼固定比率為 0.5，若 >= 我們設定的 threshold 值，則此時段（t=10）為一個固定時段。
    2. 計算所有固定時段，除以所有週末的總時段，若超過 threshold 則分類為規律人，反之為不規律人。
    """

    # 轉換 d 值為禮拜六和禮拜日，使用 %7 將日期轉換成禮拜幾，6 是禮拜六，7 是禮拜日
    df_person = df_person.copy() 
    df_person['weekday'] = df_person['d'] % 7
    weekends_data = df_person[df_person['weekday'].isin([6, 0])]  # 0 表示禮拜日，6 表示禮拜六
    
    # 建立一個字典來儲存每個 t 時段的固定比率
    fixed_ratios = {}
    
    # 統計每個 t 時段是否固定在同一地點
    for t in weekends_data['t'].unique():
        t_data = weekends_data[weekends_data['t'] == t]
        
        # 計算這個時間段有多少週末的地點是相同的
        location_counts = t_data.groupby(['d', 'x', 'y']).size().unstack(fill_value=0)
        fixed_count = (location_counts > 0).sum(axis=0).max()  # 計算最大出現次數
        
        # 固定比率 = 出現次數 / 週末的總次數
        fixed_ratio = fixed_count / len(t_data['d'].unique())
        fixed_ratios[t] = fixed_ratio
    
    # 計算固定時間段的比例
    fixed_time_slots = sum(1 for ratio in fixed_ratios.values() if ratio >= threshold)
    total_time_slots = len(fixed_ratios)
    
    # 判斷是否為規律人
    if fixed_time_slots / total_time_slots > threshold:
        return "regular"
    else:
        return "irregular"
# def home_60days(df,uid):
#     df = df[df['uid'] == uid]
#     df = df[df['d'] <= 59]
#     # 選取 t 在 44 到 12 的時間段
#     df_sleep = df[(df['t'] >= 44) | (df['t'] <= 12)]
#     sleep_location_counter = Counter(zip(df_sleep['x'], df['y']))
#     all_location_counter= Counter(zip(df['x'], df['y']))
    
#     home_most_common = sleep_location_counter.most_common(1)
#     all_most_common = all_location_counter.most_common(1)
#     # 檢查是否找到家與工作地，避免出現空列表的情況
#     if home_most_common:
#         home = home_most_common[0][0]
#     else:
#         home = all_most_common[0][0]
#     # 返回家的位置，選取最常去的地點作為家
#     print("home=",home)
#     return home
# # 假設 n 是生活圈的範圍 (以網格計算)，home_x 和 home_y 是指定 uid 的家位置
# # 找出家與指定 (home_x, home_y) 距離 n 以內的 uid
# def life_circle_uids(df, home_x, home_y, n):
#     df=df[df['d']<=59]
#     uids_within_life_circle = []

#     # 獲取所有唯一的 uid
#     uids = df['uid'].unique()

#     # 對每個 uid 找出家的位置並計算與 (home_x, home_y) 的距離
#     for uid in uids:
        
#         user_home_x, user_home_y = home_60days(df,uid)  # 找出該 uid 的家位置
        
#         # 計算家與目標家 (home_x, home_y) 的距離
#         distance = calculate_distance(user_home_x, user_home_y, home_x, home_y)
        
#         # 如果距離小於等於 n，將 uid 加入結果
#         if distance <= n:
#             uids_within_life_circle.append(uid)
    
#     return uids_within_life_circle

# # 找出生活圈內在指定時間段最熱門且離家最近的地點
# def get_popular_locations(df, home_x, home_y, n, time_start, time_end):
#     # 取得生活圈內的其他 uid
#     df=df[df['d']<=59]
#     life_circle_uids_list = life_circle_uids(df, home_x, home_y, n)

#     # 篩選出這些 uid 在指定時間段內假日去的地點
#     df_life_circle = df[(df['uid'].isin(life_circle_uids_list)) & 
#                         (df['t'] >= time_start) & (df['t'] <= time_end)]
#     df_life_circle=df_life_circle[df_life_circle['d'].apply(is_weekend)] 
#     # 計算每個地點的出現次數
#     popular_locations = df_life_circle.groupby(['x', 'y']).size().reset_index(name='count')
#     if popular_locations.empty:
#         return home_x, home_y  # 若無熱門地點，則返回家的位置

#     popular_locations['ratio'] = popular_locations['count'] / popular_locations['count'].sum()

#     # 計算每個地點與 home 之間的距離
#     popular_locations['distance'] = popular_locations.apply(
#         lambda row: calculate_distance(home_x, home_y, row['x'], row['y']), axis=1
#     )

#     # 先按 ratio 排序，然後按距離排序
#     sorted_locations = popular_locations.sort_values(by=['ratio', 'distance'], ascending=[False, True])

#     # 返回最熱門且最近的地點 (x, y)
#     nearest_popular_location = sorted_locations.iloc[0][['x', 'y']].values
#     return nearest_popular_location
def home_weekend(user_data):
    df = df[(df['t'] <= 24) & (df['t'] >= 44)]
    location_counter = Counter(zip(df['x'], df['y']))
    return location_counter.most_common(1)
    

In [6]:
def home_60days(df,uid):
    df = df[df['uid'] == uid]
    df = df[df['d'] <= 59]
    # 選取 t 在 44 到 12 的時間段
    df_sleep = df[(df['t'] >= 44) | (df['t'] <= 12)]
    sleep_location_counter = Counter(zip(df_sleep['x'], df['y']))
    all_location_counter= Counter(zip(df['x'], df['y']))
    
    home_most_common = sleep_location_counter.most_common(1)
    all_most_common = all_location_counter.most_common(1)
    # 檢查是否找到家與工作地，避免出現空列表的情況
    if home_most_common:
        home = home_most_common[0][0]
    else:
        home = all_most_common[0][0]
    # 返回家的位置，選取最常去的地點作為家
    print("home=",home)
    return home
    
# 預計算所有 uid 的家位置並儲存
def calculate_all_homes(df):
    df=df[df['d']<=59]
    homes = {}
    for uid in df['uid'].unique():
        homes[uid] = home_60days(df, uid)  # 找出該 uid 的家
    return homes

# 一次性計算所有與 (home_x, home_y) 距離小於等於 n 的 uid
def find_uids_within_distance(homes_dict, home_x, home_y, n):
    uids_within_life_circle = []
    for uid, (user_home_x, user_home_y) in homes_dict.items():
        distance = calculate_distance(user_home_x, user_home_y, home_x, home_y)
        if distance <= n:
            uids_within_life_circle.append(uid)
    return uids_within_life_circle
# 篩選符合時間段和假日的數據
def filter_data_by_time(df, time_start, time_end):
    df=df[df['d']<=59]
    return df[(df['t'] >= time_start) & (df['t'] <= time_end) & df['d'].apply(is_weekend)]
# 優化後的熱門地點查找函數
def get_popular_locations_optimized(df, home_x, home_y, n, time_start, time_end, homes_dict):
    df=df[df['d']<=59]
    life_circle_uids_list = find_uids_within_distance(homes_dict, home_x, home_y, n)
    
    # 篩選符合時間段和假日的數據
    df_filtered = filter_data_by_time(df[df['uid'].isin(life_circle_uids_list)], time_start, time_end)

    # 計算熱門地點
    popular_locations = df_filtered.groupby(['x', 'y']).size().reset_index(name='count')
    if popular_locations.empty:
        return home_x, home_y  # 若無熱門地點，則返回家的位置

    # 計算比例和距離
    popular_locations['ratio'] = popular_locations['count'] / popular_locations['count'].sum()
    popular_locations['distance'] = popular_locations.apply(
        lambda row: calculate_distance(home_x, home_y, row['x'], row['y']), axis=1
    )

    # 排序並返回最熱門且最近的地點
    sorted_locations = popular_locations.sort_values(by=['ratio', 'distance'], ascending=[False, True])
    return sorted_locations.iloc[0][['x', 'y']].values

In [ ]:
# Step 1: 取出 uid <= 3000 的資料作為測試集
df = pd.read_csv('CityC Challenge Data.csv')
df=df[df['uid']<=3000]# 取前3000筆用戶
test_df = df[df['d'].apply(is_weekend)]  # 篩選出假日資料
test_df.to_csv('answer_weekend.csv', index=False)

# Step 2: 將 60-74 日的 x 和 y 欄位設為 999（即需要預測的部分）
test_df.loc[(test_df['d'] >= 60) & (test_df['d'] <= 74), ['x', 'y']] = 999

# Step 3: 填充缺失的 x 和 y
uids = test_df['uid'].unique()

threshold = 0.5  # 規律性閾值
n = 30  # 生活圈範圍

 # 預先計算所有 uid 的家位置
homes_dict = calculate_all_homes(df)
for uid in uids:
    user_data = test_df[(test_df['uid'] == uid)]
    #判斷是否為規律人
    person_type = classify_person(user_data[user_data['d']<=59], threshold)
    # 找到該 uid 的家位置
    home = homes_dict[uid]
    home_x, home_y = home[0], home[1]
    if person_type == "regular":
        # 找出該用戶的通勤時間段
        commuting_periods = analyze_commuting_pattern(user_data[user_data['d']<=59])
        
        # 分割出早上、下午、晚上的通勤時段
        morning_commuting, afternoon_commuting, evening_commuting = split_time_slot(commuting_periods)
        
        morning_earliest = min(morning_commuting, default=12)
        morning_latest = max(morning_commuting, default=12)
        afternoon_earliest = min(afternoon_commuting, default=34)
        afternoon_latest = max(afternoon_commuting, default=34)
        evening_earliest = min(evening_commuting, default=44)
        evening_latest = max(evening_commuting, default=44)
        
        
        print(f"uid={uid}, morning: {morning_earliest}-{morning_latest}, "
            f"afternoon: {afternoon_earliest}-{afternoon_latest}, "
            f"evening: {evening_earliest}-{evening_latest}")
        
        
        # 找出該用戶的常去地點 (p1, p2, p3)
        p1, p2, p3 = find_places(user_data[user_data['d']<=59], 
                                            morning_earliest, morning_latest, 
                                            afternoon_earliest, afternoon_latest, 
                                            evening_earliest, evening_latest,home_x,home_y)
        
        print(f"uid={uid}, place1={p1}, place2={p2}, place3={p3}")

    
        # 填充 60-74 日的資料
        for i, row in user_data.iterrows():
            if 60 <= row['d'] <= 74:
                    # 早上到下午之間的工作時間段，填入 p1
                    if morning_latest <= row['t'] < afternoon_earliest:
                        test_df.at[i, 'x'], test_df.at[i, 'y'] = p1
                    
                    # 下午到晚上之間的休閒時間段，填入 p2
                    elif afternoon_latest <= row['t'] < evening_earliest:
                        test_df.at[i, 'x'], test_df.at[i, 'y'] = p2
                
                    # 晚上到早上的時間段，填入 p3
                    elif evening_latest <= row['t'] or row['t'] < morning_earliest:
                        test_df.at[i, 'x'], test_df.at[i, 'y'] = p3
                                
                    
                    elif morning_earliest <= row['t'] <morning_latest:
                        # p3到p1的通勤時間段
                        interpolated_location = linear_interpolate(p3, p1, row['t'], morning_earliest, morning_latest)
                        test_df.at[i, 'x'], test_df.at[i, 'y'] = interpolated_location
                        
                    elif afternoon_earliest <= row['t'] < afternoon_latest:
                        # p1到p2的通勤時間段
                        interpolated_location = linear_interpolate(p1, p2, row['t'], afternoon_earliest, afternoon_latest)
                        test_df.at[i, 'x'], test_df.at[i, 'y'] = interpolated_location
                    elif evening_earliest <= row['t'] < evening_latest:
                        # p2到p3的通勤時間段
                        interpolated_location = linear_interpolate(p2, p3, row['t'], evening_earliest, evening_latest)
                        test_df.at[i, 'x'], test_df.at[i, 'y'] = interpolated_location

    else:  # 不規律人
        # 遍歷指定日期範圍的數據進行預測
        for i, row in user_data.iterrows():
            if 60 <= row['d'] <= 74:
                if 24 <= row['t'] <= 34:
                    # 填充最熱門的下午地點
                    afternoon_location = get_popular_locations_optimized(df, home_x, home_y, n, 24, 34, homes_dict)
                    test_df.at[i, 'x'], test_df.at[i, 'y'] = afternoon_location
                elif 34 <= row['t'] <= 44:
                    # 填充最熱門的傍晚地點
                    evening_location = get_popular_locations_optimized(df, home_x, home_y, n, 34, 44, homes_dict)
                    test_df.at[i, 'x'], test_df.at[i, 'y'] = evening_location
                else:
                    # 填充家的位置
                    test_df.at[i, 'x'], test_df.at[i, 'y'] = home_x, home_y
# Step 4: 將結果保存成 CSV 檔案
test_df.to_csv('fill_weekend.csv', index=False)

print("預測結果已保存到 fill_weekend.csv")

In [ ]:
import geobleu
from geobleu import geobleu
import pandas as pd

# 讀取 answer.csv 和 fill.csv
answer_df = pd.read_csv('answer.csv')
fill_df = pd.read_csv('fill.csv')

# 初始化變量存儲所有分數
total_similarity = 0
count = 0

# 假設這兩個檔案都有 'uid', 'd', 't', 'x', 'y' 五個欄位
uids = answer_df['uid'].unique()

for uid in uids:
    # 取出每個 uid 的資料
    answer_user_data = answer_df[answer_df['uid'] == uid]
    fill_user_data = fill_df[fill_df['uid'] == uid]
    
    # 將時間順序與座標結合，形成 (d, t, x, y) 格式的列表
    reference_coords = list(zip(answer_user_data['d'], answer_user_data['t'], answer_user_data['x'], answer_user_data['y']))
    generated_coords = list(zip(fill_user_data['d'], fill_user_data['t'], fill_user_data['x'], fill_user_data['y']))
    
    # 計算該使用者的 GeoBLEU 分數
    similarity = geobleu.calc_geobleu(generated_coords, reference_coords, processes=3)
    dtw_val = geobleu.calc_dtw(generated_coords, reference_coords, processes=3)

    # 累加分數
    total_similarity += similarity
    total_dtw += dtw_val
    count += 1

# 計算平均分數
average_similarity = total_similarity / count if count > 0 else 0
avg_dtw = total_dtw / count if count > 0 else 0
print(f'Average GeoBLEU score: {average_similarity}')
print("dtw: {}".format(dtw_val))